In [11]:
import json
import requests
import pandas as pd
import xlrd
import urllib.parse

In [25]:
file = "../mockup_data/Pages/Sample clinical data_July 2022.xlsx"
writer = pd.ExcelWriter(file+'_perols.xlsx', engine='xlsxwriter')
dfsource = pd.read_excel("../mockup_data/Pages/Sample clinical data_July 2022.xlsx", header=0)
dfsource

,ID Lab site,ID clinical site,DOB,Age,Gender,Pathology,Localization,IDH1/2 status,MGMT status,ECOG,...,doses bevacizumab,Best response,pseudo-progression (1=yes 0=no),œdema,PP or œdema,progression (1=yes 0=no),date of progression,months PFS,deceased (1=yes 0=no),months OS
0,Ge 829,DAG,1953-05-16,62,F,GBM,parietal R,wt,UnMeth,1,...,12,PD,0,1,1,1,2013-11-11,3,1,19
1,Ge 835,EAG,1964-06-18,50,M,GBM,temporal D,wt,UnMeth,0,...,0,PD,1,1,1,1,2014-01-21,6,1,10
2,Ge 849,FAG,1957-12-15,57,F,GBM,frontal R,wt,UnMeth,0,...,9,PD,1,0,1,1,2014-09-02,11,1,18
3,Ge 852,GAG,1960-03-22,55,M,GBM,fronto insular L,wt,UnMeth,0,...,16,SD,0,1,1,1,2014-10-01,11,1,21
4,Ge 882,IAG,1948-05-06,67,M,GBM,fronto insular R,wt,UnMeth,0,...,4,PD,0,1,1,1,2014-12-22,18,1,24
5,Ge 893*,KAG,1958-07-12,56,M,GBM,temporal R,wt,Meth,0,...,0,SD,0,0,0,1,2015-02-04,26,0,44
6,Ge 901,LAG,1991-06-09,23,F,AIII,fronto temporal insular R,mut,UnMeth,0,...,29,SD,1,0,1,1,2016-07-18,17,1,37
7,Ge 904,MAG,1949-10-13,65,M,GBM,parieto occipital L,wt,Meth,0,...,6,PR,1,1,1,1,2016-12-12,29,1,35
8,Ge 939,PAG,1972-04-10,43,F,AIII,temporal R,wt,UnMeth,1,...,42,PD,1,1,1,1,2015-06-12,6,1,32
9,Ge 941*,OAG,1954-01-07,61,M,GBM,frontal L,mut,Meth,0,...,0,CR,1,0,1,0,NaT,38,0,38


In [26]:
for field in dfsource.columns:
    df = pd.DataFrame(columns=['id', 'ontology_name', 'iri', 'obo_id', 'label', 'description',  'type', 'is_defining_ontologyy' ])
    url = "https://www.ebi.ac.uk/ols/api/search?q="+urllib.parse.quote(field)+"&queryFields=label,synonym"
    responses = json.loads(requests.get(url=url).text)
    for response in responses['response']['docs']:
        if "id" in response.keys():
            id = response["id"]
        else:
            id = None
        if "ontology_name" in response.keys():
            ontology_name = response["ontology_name"]
        else:
            ontology_name = None
        if "iri" in response.keys():
            iri = response["iri"]
        else:
            iri = None
        if "obo_id" in response.keys():
            obo_id = response["obo_id"]
        else:
            obo_id = None
        if "label" in response.keys():
            label = response["label"]
        else:
            label = None
        if "description" in response.keys():
            description = "\n".join(response["description"])
        else:
            description = None
        if "type" in response.keys():
            type = response["type"]
        else:
            type = None
        if "is_defining_ontology" in response.keys():
            is_defining_ontology = response["is_defining_ontology"]
        else:
            is_defining_ontology = None
        df.loc[len(df.index)] = [id, ontology_name, iri, obo_id, label, description,  type, is_defining_ontology ]
    if len(field) >31:
        sheetname = field[:27]+"..."
    else:
        sheetname = field
    df.to_excel(writer, sheet_name=sheetname.replace("/",""), index=False)
    print(sheetname)
writer.close()


ID Lab site
ID clinical site
DOB
Age 
Gender
Pathology
Localization 
IDH1/2 status
MGMT status
ECOG
Nature of initial resection 
lymphocyte counts at V1 in G/L
cummulative dose preV1 ster...
steroid use after V1
cummulative steroid dose in mg
 Cycles adj TMZ
doses bevacizumab
Best response
pseudo-progression (1=yes 0=no)
œdema
PP or œdema
progression (1=yes 0=no)
date of progression
months PFS
deceased (1=yes 0=no)
months OS
